In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import Video
from IPython.display import Audio

In [ ]:
def get_interpolated(array, index):
    if not hasattr(array, "__len__"): return array # if scalar
    return (1 - (index[0] % 1)) * get_interpolated(array[int(np.floor(index[0]))], index[1:]) + (index[0] % 1) * get_interpolated(array[int(np.ceil(index[0]))], index[1:])

In [ ]:
def calculate_next_psi(psi, dt, potential):
    n = psi.shape[0]
    
    # potential-part
    next_psi = [[psi[i, j] * np.exp(-1j * dt * potential[i, j]) for j in range(n)] for i in range(n)]

    next_psi = np.fft.fft2(next_psi)
    
    for i in range(n):
        for j in range(n):
            k = 2*np.pi * min(i, n-i)
            l = 2*np.pi * min(j, n-j)
            theta = (k*k + l*l) * dt
            next_psi[i, j] *= np.exp(1j * theta)
    
    next_psi = np.fft.ifft2(next_psi)
    return next_psi

In [ ]:
def gaussian(x, y, n, offset, width):
    x = (x - n/2.0) / (n/2.0) - offset[0]
    y = (y - n/2.0) / (n/2.0) - offset[1]
    return np.exp(-(x*x + y*y) / (width*width)) + 0j

In [ ]:
def parabolar(x, y, n, offset, factor):
    x = (x - n/2.0) / (n/2.0) - offset[0]
    y = (y - n/2.0) / (n/2.0) - offset[1]
    return factor * (x*x + y*y)
    

In [ ]:
parabolar(64, 0, 128, [0, 0], 1)

In [ ]:
def circle(rad, radius, offset, n):
    return (radius * np.array([np.cos(rad), np.sin(rad)]) + offset) * n//2 + n//2

In [ ]:
np.concatenate(([5], circle(0, 1, 0, 128)), axis=0)

In [ ]:
%rm test.mp4

n = 128
fps = 12
duration = 5

samplerate = 44100
frequency = 110
interest_radius = 0.5
interest_center = [0, 0]

frames = [] # for storing the generated images
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('test.mp4', fourcc, fps, (800, 600))

psi = np.array([[gaussian(x, y, n, offset=[-0.5, 0.0], width=0.2) for x in range(n)] for y in range(n)])
potential = np.array([[parabolar(x, y, n, offset=[0,0], factor=100) for x in range(n)] for y in range(n)])
frames.append(psi)

#plt.pcolormesh(pow(np.abs(frames[0]), 2.0/3.0), cmap='inferno', vmin=0, vmax=1)
plt.pcolormesh(potential)
plt.colorbar()
plt.show()


for i in range(fps*duration):
    for timestep in range(1):
        psi = calculate_next_psi(psi, 0.0005, potential)
    frames.append(psi)

print("Finished simulation")

# Sonification
rad_per_sample = 2 * np.pi * frequency / samplerate
audio = [np.square(np.abs(get_interpolated(frames, np.concatenate((np.array([i * fps/samplerate]), circle(rad_per_sample * i, interest_radius, interest_center, n)), axis=0)))) for i in range(samplerate*duration)]

print("Finished sonicifaction")

'''
for i in range(len(frames)):
    plt.figure(figsize=(8, 6))
    plot = plt.pcolormesh(pow(np.abs(frames[i]), 2.0/3.0), cmap='inferno', vmin=0, vmax=1, animated=True)
    plt.colorbar()
    plt.title('psi')

    # Save to temporary file
    plt.savefig('temp_frame.png')
    plt.close()

    # Read the image and add to the video
    img = cv2.imread('temp_frame.png')
    video.write(img)

video.release()
'''

In [ ]:
Audio(audio, rate=samplerate)